In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, make_scorer, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('data/Train.csv')
removed_features = ['Place_ID X Date', 'Date', 'target_min','target_max', 'target_variance', 'target_count']
train = train.drop(columns=removed_features)
len(train.columns)

76

In [4]:
train.shape

(30557, 76)

In [5]:
#train = train[train['target'] <= 300]

In [6]:
target = train['target']
train = train.drop('target', axis=1)
len(train.columns)

75

In [7]:
test = pd.read_csv('data/Test.csv')
removed_features = ['Place_ID X Date', 'Date']
test = test.drop(columns=removed_features)
len(test.columns)

75

In [8]:
train.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [9]:
numeric_cols = train.select_dtypes(include='number').columns
train[numeric_cols] = train.groupby('Place_ID')[numeric_cols].transform(lambda x: x.fillna(x.mean()))
train[numeric_cols] = train[numeric_cols].fillna(train[numeric_cols].mean())

In [10]:
numeric_cols = test.select_dtypes(include='number').columns
test[numeric_cols] = test.groupby('Place_ID')[numeric_cols].transform(lambda x: x.fillna(x.mean()))

In [11]:
# Step 1: Detect skewed columns (absolute skewness > 0.75)
skewness = train[numeric_cols].skew()
skewed_cols = skewness[abs(skewness) > 0.75].index.tolist()

# Step 2: Apply PowerTransformer (Yeo-Johnson) to reduce skewness
pt = PowerTransformer(method='yeo-johnson')
train[skewed_cols] = pt.fit_transform(train[skewed_cols])
test[skewed_cols] = pt.transform(test[skewed_cols])

In [12]:
# Select numeric columns
numeric_cols_ = train.select_dtypes(include='number').columns

# Create a scaler and fit on train
scaler = StandardScaler()
train[numeric_cols] = scaler.fit_transform(train[numeric_cols])

# Apply the same transformation to test
test[numeric_cols] = scaler.transform(test[numeric_cols])

In [13]:
oo = ['Place_ID']
train = train.drop(columns=oo)
test = test.drop(columns=oo)

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Assuming your data is in a pandas DataFrame
# X should be your features DataFrame and y should be your target Series

X = train
y = target

# Initialize base models
xgb = XGBRegressor(    n_estimators=10000,       # Equivalent to 'iterations' in CatBoost
    learning_rate=0.06,      # Same as CatBoost
    max_depth=8,              # Same as 'depth' in CatBoost
    objective='reg:squarederror',  # Default regression objective
    random_state=42,          # Same as 'random_seed' in CatBoost
    subsample=0.7,            # Similar to bagging (adjust as needed)
    colsample_bytree=0.9,     # Feature subsampling
    #early_stopping_rounds=300, # Similar to 'od_wait' (stops if no improvement)
    eval_metric='rmse',       # Same as CatBoost
    n_jobs=-1 
    )
cat = CatBoostRegressor(n_estimators=10000,       # Equivalent to 'iterations' in CatBoost
        learning_rate=0.05,      # Same as CatBoost
        max_depth=5,              # Same as 'depth' in CatBoost
        random_state=42,
        verbose=0,                 # Suppress output during training
        loss_function='RMSE',
        random_strength = 10,
        l2_leaf_reg = 5,
        border_count = 32,
        bagging_temperature = 1,
        od_type='Iter',
        metric_period = 50,
        od_wait=300 )
rf = RandomForestRegressor(n_estimators=10000,
        max_depth=8,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='sqrt',
        bootstrap=True,
        random_state=42,
        n_jobs=-1)

# Initialize meta-model
meta_model = LinearRegression()

# Initialize KFold
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# Create array for holding out-of-fold predictions
meta_features = np.zeros((X.shape[0], 3))  # 3 columns for 3 models

# Step 1: Generate out-of-fold predictions for each base model
for i, model in enumerate([xgb, cat, rf]):
    for train_idx, val_idx in kf.split(X, y):
        # Convert indices to actual data - important for DataFrames
        X_train = X.iloc[train_idx]
        X_val = X.iloc[val_idx]
        y_train = y.iloc[train_idx]
        y_val = y.iloc[val_idx]
        
        # Fit the model
        model.fit(X_train, y_train)
        
        # Get predictions for validation set
        preds = model.predict(X_val)
        meta_features[val_idx, i] = preds

# Step 2: Train the meta-model on the out-of-fold predictions
meta_model.fit(meta_features, y)

# Step 3: Train base models on full training data for final prediction
xgb.fit(X, y)
cat.fit(X, y)
rf.fit(X, y)


'''
# Get base model predictions on test set
xgb_test_preds = xgb.predict(X_test)
cat_test_preds = cat.predict(X_test)
rf_test_preds = rf.predict(X_test)

# Create meta-features for test set
meta_features_test = np.column_stack((xgb_test_preds, cat_test_preds, rf_test_preds))

# Get stacked predictions
stacked_preds = meta_model.predict(meta_features_test)

# Evaluate
mse = mean_squared_error(y_test, stacked_preds)
print(f"Stacked Model MSE: {mse:.4f}")

# Compare with individual models
for name, preds in zip(['XGBoost', 'CatBoost', 'Random Forest'], 
                       [xgb_test_preds, cat_test_preds, rf_test_preds]):
    mse = mean_squared_error(y_test, preds)
    print(f"{name} MSE: {mse:.4f}")
'''

'\n# Get base model predictions on test set\nxgb_test_preds = xgb.predict(X_test)\ncat_test_preds = cat.predict(X_test)\nrf_test_preds = rf.predict(X_test)\n\n# Create meta-features for test set\nmeta_features_test = np.column_stack((xgb_test_preds, cat_test_preds, rf_test_preds))\n\n# Get stacked predictions\nstacked_preds = meta_model.predict(meta_features_test)\n\n# Evaluate\nmse = mean_squared_error(y_test, stacked_preds)\nprint(f"Stacked Model MSE: {mse:.4f}")\n\n# Compare with individual models\nfor name, preds in zip([\'XGBoost\', \'CatBoost\', \'Random Forest\'], \n                       [xgb_test_preds, cat_test_preds, rf_test_preds]):\n    mse = mean_squared_error(y_test, preds)\n    print(f"{name} MSE: {mse:.4f}")\n'

In [15]:
# Get base model predictions on test set
xgb_test_preds = xgb.predict(test)
cat_test_preds = cat.predict(test)
rf_test_preds = rf.predict(test)

# Create meta-features for test set
meta_features_test = np.column_stack((xgb_test_preds, cat_test_preds, rf_test_preds))

# Get stacked predictions
stacked_preds = meta_model.predict(meta_features_test)

In [16]:
dddd = pd.read_csv('data/Test.csv')
dddd['target'] = xgb_test_preds
df_upload = dddd[['Place_ID X Date', 'target']]
df_upload.to_csv('data/Submission_xgb_test_preds.csv', index=False)

In [17]:
dddd = pd.read_csv('data/Test.csv')
dddd['target'] = cat_test_preds
df_upload = dddd[['Place_ID X Date', 'target']]
df_upload.to_csv('data/Submission_cat_test_preds.csv', index=False)

In [18]:
dddd = pd.read_csv('data/Test.csv')
dddd['target'] = rf_test_preds
df_upload = dddd[['Place_ID X Date', 'target']]
df_upload.to_csv('data/Submission_rf_test_preds.csv', index=False)

In [19]:
dddd = pd.read_csv('data/Test.csv')
dddd['target'] = stacked_preds
df_upload = dddd[['Place_ID X Date', 'target']]
df_upload.to_csv('data/Submission_stacked_preds.csv', index=False)

In [20]:
X/0

,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,-inf,-inf,inf,inf,inf,-inf,-inf,inf,-inf,-inf,...,inf,inf,-inf,inf,inf,inf,inf,inf,-inf,inf
1,inf,-inf,inf,inf,inf,-inf,-inf,inf,inf,-inf,...,inf,inf,-inf,inf,inf,inf,inf,inf,-inf,inf
2,inf,-inf,inf,inf,inf,inf,-inf,inf,inf,-inf,...,inf,inf,-inf,inf,inf,-inf,-inf,inf,inf,inf
3,-inf,-inf,-inf,inf,inf,inf,inf,inf,inf,-inf,...,-inf,inf,-inf,inf,inf,-inf,-inf,inf,inf,inf
4,inf,-inf,inf,inf,inf,inf,-inf,-inf,inf,-inf,...,-inf,inf,-inf,inf,inf,-inf,-inf,inf,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30552,-inf,-inf,-inf,-inf,-inf,-inf,-inf,inf,-inf,-inf,...,inf,inf,inf,inf,inf,inf,-inf,inf,-inf,inf
30553,-inf,inf,-inf,-inf,-inf,inf,-inf,inf,-inf,-inf,...,inf,-inf,inf,inf,inf,inf,-inf,inf,-inf,inf
30554,-inf,-inf,-inf,-inf,-inf,inf,-inf,-inf,-inf,-inf,...,-inf,-inf,inf,inf,inf,inf,-inf,inf,-inf,inf
30555,inf,-inf,inf,inf,inf,-inf,inf,-inf,-inf,-inf,...,-inf,-inf,-inf,inf,inf,inf,-inf,inf,-inf,inf


In [21]:
XGB = XGBRegressor(
    n_estimators=30000,       # Equivalent to 'iterations' in CatBoost
    learning_rate=0.06,      # Same as CatBoost
    max_depth=8,              # Same as 'depth' in CatBoost
    objective='reg:squarederror',  # Default regression objective
    random_state=42,          # Same as 'random_seed' in CatBoost
    subsample=0.7,            # Similar to bagging (adjust as needed)
    colsample_bytree=0.9,     # Feature subsampling
    #early_stopping_rounds=300, # Similar to 'od_wait' (stops if no improvement)
    eval_metric='rmse',       # Same as CatBoost
    n_jobs=-1                 # Use all CPU cores
)

In [22]:
XGB.fit(train, target)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.06, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=30000, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [23]:
y_pred_upload = XGB.predict(test)
dddd = pd.read_csv('data/Test.csv')
dddd['target'] = y_pred_upload
df_upload = dddd[['Place_ID X Date', 'target']]
df_upload.to_csv('data/Submission_20250802_2.csv', index=False)

In [24]:
feature_importance_df = pd.DataFrame({
    'Feature': train.columns,
    'Importance': XGB.feature_importances_
})
feature_importance_df.sort_values(by='Importance', ascending=False)


,Feature,Importance
25,L3_CO_CO_column_number_density,0.097794
53,L3_AER_AI_sensor_altitude,0.096150
39,L3_HCHO_tropospheric_HCHO_column_number_density,0.054412
7,L3_NO2_NO2_slant_column_number_density,0.048577
73,L3_CH4_solar_zenith_angle,0.037942
...,...,...
22,L3_O3_sensor_zenith_angle,0.003017
48,L3_CLOUD_sensor_zenith_angle,0.002978
21,L3_O3_sensor_azimuth_angle,0.002968
11,L3_NO2_sensor_azimuth_angle,0.002962
